In [3]:
%load_ext autoreload
%autoreload 2
%run Datasets.ipynb
import numpy as np
from keras import Input, Model
from keras.src.layers import Dense, BatchNormalization, GaussianNoise



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
code_rate = 4/7
neurons = 7

entrada = Input(shape=(SIZE,))
encoder1 = Dense(SIZE, activation='relu')(entrada)
encoder2 = Dense(neurons, activation='linear')(encoder1)
encoder3 = BatchNormalization()(encoder2)


Eb_treinamento = 5.01187 * code_rate

noise = GaussianNoise(np.sqrt(1/(2* Eb_treinamento)))(encoder3)

decoder1 = Dense(SIZE, activation='relu')(noise)
decoder2 = Dense(SIZE, activation='softmax')(decoder1)

autoencoder = Model(entrada, decoder2)
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
autoencoder.fit(
    training_data,
    training_data,
    epochs=20,
    batch_size=300,
    validation_data=(validation_data, validation_data),
)

# Com o modelo treinado, separa em encoder e decoder
encoder = Model(entrada, encoder3)

entrada_encoder = Input(shape=(7,))
decoder_layer1 = autoencoder.layers[-2](entrada_encoder)
decoder_layer2 = autoencoder.layers[-1](decoder_layer1)
decoder = Model(entrada_encoder, decoder_layer2)

Epoch 1/20
34/34 [==============================] - 4s 16ms/step - loss: 1.5694 - accuracy: 0.1204 - val_loss: 1.3760 - val_accuracy: 0.2660
Epoch 2/20
34/34 [==============================] - 0s 7ms/step - loss: 1.3690 - accuracy: 0.1782 - val_loss: 1.3416 - val_accuracy: 0.2660
Epoch 3/20
34/34 [==============================] - 0s 7ms/step - loss: 1.2097 - accuracy: 0.3728 - val_loss: 1.2716 - val_accuracy: 0.5040
Epoch 4/20
34/34 [==============================] - 0s 6ms/step - loss: 1.0491 - accuracy: 0.4792 - val_loss: 1.1624 - val_accuracy: 0.5040
Epoch 5/20
34/34 [==============================] - 0s 6ms/step - loss: 0.8773 - accuracy: 0.5282 - val_loss: 1.0276 - val_accuracy: 0.7530
Epoch 6/20
34/34 [==============================] - 0s 6ms/step - loss: 0.7308 - accuracy: 0.7075 - val_loss: 0.8836 - val_accuracy: 0.7530
Epoch 7/20
34/34 [==============================] - 0s 6ms/step - loss: 0.6198 - accuracy: 0.7912 - val_loss: 0.7592 - val_accuracy: 0.7340
Epoch 8/20
34/34 [=

In [ ]:
Eb_dB_values = np.arange(-4, 8, 1)
ber_autoencoder = []

for i in range(0, len(Eb_dB_values)):
    
    Eb = 10**(Eb_dB_values[i]/10)
    quantidade_amostras = test_length / SIZE
    noise = np.sqrt((1/(2*code_rate*Eb))) * np.random.randn(test_length, 7)    
    
    # codifica para 7 bits
    codificado = encoder.predict(testing_data)
    
    # add ruido
    codificado_com_ruido = codificado + noise
    
    # corrige com o decoder
    corrigido = decoder.predict(codificado_com_ruido)
    
    digital = np.argmax(corrigido, axis = 1)
    
    erros = digital != testing_label
    erros = erros.astype(int).sum()
    calculated_ber = erros/test_length
    ber_autoencoder.append(erros/test_length)

    print('SNR ', Eb_dB_values[i], 'BER ', calculated_ber)
    
    


3125/3125 [==============================] - 8s 3ms/step
SNR  -4 BER  0.18296
3125/3125 [==============================] - 7s 2ms/step
SNR  -3 BER  0.14735
3125/3125 [==============================] - 7s 2ms/step
SNR  -2 BER  0.114
3125/3125 [==============================] - 6s 2ms/step
SNR  -1 BER  0.08646
2127/3125 [===================>..........] - ETA: 1s

In [ ]:
from matplotlib import pyplot as plt

plt.plot(Eb_dB_values, ber_autoencoder, 'bo')
plt.yscale('log')
plt.show()